<a href="https://colab.research.google.com/github/pabloalfaro/drl-grid/blob/main/pruebas/dqn/01_DQN_lunarlander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning para _lunar lander_

[**Juan Gómez Romero**](https://decsai.ugr.es/~jgomez)  
Departamento de Ciencias de la Computación e Inteligencia Artificial  
Universidad de Granada  
This work is licensed under the [GNU General Public License v3.0](https://choosealicense.com/licenses/gpl-3.0/).

---
Ejemplo basado en:
> Udacity (2019) Deep Reinforcement Learning Course. Disponible en [GitHub](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn).

Comprobar si está ejecutando en Google Colaboratory.

In [6]:
!pip install torch torchvision pillow gym requests sklearn matplotlib collections-extended numpy box2d-py gym[Box_2D]

In [7]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %cd "/content/drive/MyDrive/ECI/eci2019-DRL/Tema 4 - Aprendizaje Profundo por Refuerzo/code/dqn"
    render = False
else:
    render = True

/content/drive/MyDrive/ECI/eci2019-DRL/Tema 4 - Aprendizaje Profundo por Refuerzo/code/dqn


## Explorar entorno virtual

En este ejercicio utilizamos el entorno virtual [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) de [OpenAI](https://openai.com). 

![](https://github.com/jgromero/eci2019-DRL/blob/master/Tema%204%20-%20Aprendizaje%20Profundo%20por%20Refuerzo/code/dqn/lunarlander.gif?raw=true)

En primer lugar, vamos a explorar cómo funciona este entorno.

In [9]:
import gym

Crear entorno:

In [5]:
env = gym.make("LunarLander-v2")

Cada estado es una tupla de 8 elementos. Los dos primeros valores corresponden a la posición del módulo.

El agente puede realizar 4 acciones:

```
    NADA = 0
    ACTIVAR MOTOR DE ORIENTACION IZQUIERDO = 1
    ACTIVAR MOTOR PRINCIPAL = 2
    ACTIVAR MOTOR DE ORIENTACION DERECHO = 3
```

El objetivo en [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) es conseguir aterrizar el módulo lunar en el espacio designado ($+200$). La puntuación que se obtiene por moverse desde la parte superior de la pantalla hasta la zona de aterrizaje está en torno a $[100, 140]$. Si el módulo se desvía de la zona de aterrizaje pierde puntuación. El episodio termina si el módulo se estrella ($-100$) o se detiene ($+100$). Cada para que contacte con el suelo aumenta la puntuación ($+10$). Activar el motor principal supone una penalización ($-0.3$ por frame). 

In [10]:
print(env.observation_space)
print(env.action_space)
print(env.reward_range)

(0.0, 20.0)


In [11]:
print(env.observation_space)
print(env.action_space.sample())

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - Change the bus of the following element(s):
	 	 - Switch bus of line (extremity) id 17 [on substation 5]
	 	 - Switch bus of generator id 2 [on substation 5]
	 	 - Switch bus of load id 5 [on substation 5]
	 - NOT force any particular bus configuration


Implementación de un agente con comportamiento sin entrenar utilizando la clase [`Agent`](dqn_agent.py).

In [24]:
pprint(state)

OrderedDict([('_shunt_bus', array([1], dtype=int32)),
             ('_shunt_p', array([0.], dtype=float32)),
             ('_shunt_q', array([-17.828289], dtype=float32)),
             ('_shunt_v', array([0.20148437], dtype=float32)),
             ('a_ex',
              array([1.8238358e+02, 1.7948483e+02, 3.1578791e+04, 4.0613402e+04,
       1.5287653e+04, 4.1530352e+03, 1.7193543e+04, 1.0401473e+02,
       2.0465733e+02, 1.8260760e+02, 1.5681223e+02, 1.3504549e+02,
       2.2531211e+04, 2.4254668e+04, 5.5535344e+04, 1.2402637e+03,
       5.1626156e+04, 1.4019395e+05, 3.5587805e+02, 9.2169969e+04],
      dtype=float32)),
             ('a_or',
              array([  189.77327,   169.29967, 31578.791  , 40613.402  , 15287.653  ,
        4153.035  , 17193.543  ,   108.55819,   197.10118,   173.4902 ,
         151.91942,   135.04549, 22531.21   , 24254.668  , 55535.344  ,
         131.51323,    82.08723,   231.76239,   305.03833,  1369.3823 ],
      dtype=float32)),
             ('actual_

In [19]:
from dqn_agent import Agent

agent = Agent(state_size=8, action_size=4, seed=0)

state = env.reset()
for j in range(200):
    action = agent.act(state)
    if render: env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 
        
env.close()

TypeError: ignored

## Algoritmo
A continuación se proporciona una implementación genérica del algoritmo Deep Q-Learning (DQN) y su aplicación a [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/).

Se considera que el entorno [LunarLander-v2](https://gym.openai.com/envs/LunarLander-v2/) está resuelto cuando se obtienen más de $200$ puntos de media durante 100 episodios consecutivos.

In [9]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): numero maximo de episodios de entrenamiento (n_episodios)
        max_t (int): numero maximo de pasos por episodio (n_entrenamiento)
        eps_start (float): valor inicial de epsilon
        eps_end (float): valor final de epsilon
        eps_decay (float): factor de multiplicacion (por episodio) de epsilon
    """
    scores = []                        # puntuaciones de cada episodio
    scores_window = deque(maxlen=100)  # puntuaciones de los ultimos 100 episodios
    eps = eps_start                    # inicializar epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            
            # elegir accion At con politica e-greedy
            action = agent.act(state, eps)
            
            # aplicar At y obtener Rt+1, St+1
            next_state, reward, done, _ = env.step(action)
            
            # almacenar <St, At, Rt+1, St+1>
            agent.memory.add(state, action, reward, next_state, done)
            
            # train & update
            agent.step(state, action, reward, next_state, done)
            
            # avanzar estado
            state = next_state
            score += reward
            
            if done:
                break 

        scores_window.append(score)       # guardar ultima puntuacion
        scores.append(score)              # guardar ultima puntuacion
        eps = max(eps_end, eps_decay*eps) # reducir epsilon
        
        print('\rEpisodio {}\tPuntuacion media (ultimos {:d}): {:.2f}'.format(i_episode, 100, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisodio {}\tPuntuacion media ({:d} anteriores): {:.2f}'.format(i_episode, 100, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nProblema resuelto en {:d} episodios!\tPuntuacion media (ultimos {:d}): {:.2f}'.format(i_episode-100, 100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth') # guardar pesos de agente entrenado
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Puntuacion')
plt.xlabel('Episodio #')
plt.show()

NameError: ignored

### Visualizar agente entrenado

Podemos visualizar el comportamiento del agente entrenado cargando los pesos del fichero donde se han almacenado. (Solo en entorno local, no en Google Collaboratory.)

In [11]:
# cargar pesos del fichero `checkpoint.pth`
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(3):
    state = env.reset()
    for j in range(200):
        action = agent.act(state)
        if render: env.render()
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()

---

### Ampliación

Trabajo de grid2op
---

In [12]:
 !pip install grid2op[optional]  # Para ejecutar el código en Colab

     |████████████████████████████████| 11.9 MB 73 kB/s 
     |████████████████████████████████| 5.8 MB 23.3 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 112 kB 67.9 MB/s 
     |████████████████████████████████| 1.5 MB 34.8 MB/s 
     |████████████████████████████████| 20.6 MB 1.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.5 MB/s 
     |████████████████████████████████| 296 kB 47.8 MB/s 
     |████████████████████████████████| 148 kB 44.8 MB/s 
     |████████████████████████████████| 3.2 MB 47.5 MB/s 
  Created wheel for grid2op: filename=Grid2Op-1.6.2-py3-none-any.whl size=12150152 sha256=47375aa66d7ccd6de39f048388d56d5ba02a1dc733c008fddacd37cec46e286f
  Stored in directory: /root/.cache/pip/wheels/d0/12/bf/e2cb4f3b2fd2a85af617053e46e272de1866c647f6640f3ad4
  Created wheel for pandapower: filename=pandapower-2.7.0-py3-none-any.whl size=5777359 sha256=287a80ae351984764db3a687e53c8d9b6a2fdcdb99a720d3cb8788d2e98a1b4f
 

In [3]:
 !pip install jyquickhelper # Para ejecutar el código en Colab

In [1]:
import os
import sys
import grid2op

In [2]:
# import the usefull classes
import numpy as np
import shutil
from tqdm.notebook import tqdm  # for easy progress bar

from grid2op import make
from grid2op.Agent import DoNothingAgent

from grid2op.Reward import GameplayReward, L2RPNReward

max_iter = 50 # Se consideran 50 iteraciones para que sea más rápido
train_iter = 50
max_eval_step = 20
env_name = "rte_case14_redisp"
env = make(env_name, test=False, reward_class=L2RPNReward)
env.seed(0)  # Una semilla para que los agentes sean reproducibles

(1537364731,
 (209652396, 2087557356),
 (398764591,),
 (924231285,),
 (1478610112,),
 ((441365315,), (633163265,)),
 (1537364731,))

In [4]:
scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Puntuacion')
plt.xlabel('Episodio #')
plt.show()

NameError: ignored

In [14]:
!pip install git+https://github.com/DLR-RM/stable-baselines3

  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-udq0glov
  Running command git clone -q https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-udq0glov
  Created wheel for stable-baselines3: filename=stable_baselines3-1.2.0a1-py3-none-any.whl size=160853 sha256=b5b5ea7b44c01972ecf7136161cb53dfcf7579bf422a7dd8567b085d9f2b230a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ub6tl625/wheels/2b/88/65/5d0cb266b061107af8c518096240bea8578e9843716f79e4da
Successfully built stable-baselines3


In [15]:
from grid2op.Reward import L2RPNReward

from grid2op.gym_compat import GymEnv, MultiDiscreteActSpace

from stable_baselines3.common.policies import BasePolicy
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_checker import check_env
from pprint import pprint #Para ver los diccionarios con una estructura más elegante de la que ofrece print convencional

In [16]:
gym_env = GymEnv(env)
# Aunque el espacio de observaciones y acciones ya es correcto, Stable Baselines 3 no tiene soporte para espacio de acciones mixtos basados en diccionarios
# Por lo que no nos queda más remedio que discretizar todo el espacio de acciones (leer docstring de https://github.com/rte-france/Grid2Op/blob/master/grid2op/gym_compat/multidiscrete_gym_actspace.py)
print("Espacio de acciones original: {}".format(gym_env.action_space))
gym_env.action_space = MultiDiscreteActSpace(gym_env.init_env.action_space)
print("Espacio de acciones discretizado: {}".format(gym_env.action_space))

Espacio de acciones original: Dict(change_bus:MultiBinary(56), change_line_status:MultiBinary(20), redispatch:Box(-10.0, 10.0, (5,), float32), set_bus:Box(-1, 2, (56,), int32), set_line_status:Box(-1, 1, (20,), int32))
Espacio de acciones discretizado: MultiDiscrete([2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 7 7 1 1 7 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3])


/usr/local/lib/python3.7/dist-packages/grid2op/gym_compat/multidiscrete_gym_actspace.py:170: UserWarning: The class "MultiDiscreteActSpace" should mainly be used to consider only discrete actions (eg. set_line_status, set_bus or change_bus). Though it is possible to use "redispatch" when building it, be aware that this continuous action will be treated as discrete by splitting it into bins. Consider using the "BoxGymActSpace" for these attributes.
  warnings.warn(f"The class \"MultiDiscreteActSpace\" should mainly be used to consider only discrete "


In [17]:
check_env(gym_env)

In [18]:
env = gym_env